In [4]:
# загружаем данные
import pandas as pd
import numpy as np

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv(
    ".\lastfm_small\lastfm_small.tsv",
    sep="\t",
    header=None,
    names=col_names
)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [6]:
# заполняем пустые значения
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [7]:
data.describe()

,total-plays,user_id,artist_id
count,1000000.000000,1000000.000000,1000000.000000
mean,216.606950,10232.925996,33678.492236
std,604.378024,5912.022447,19230.330182
min,1.000000,1.000000,1.000000
25%,34.000000,5118.000000,17298.000000
50%,94.000000,10237.000000,34544.000000
75%,225.000000,15347.000000,49488.000000
max,135392.000000,20465.000000,66799.000000


In [16]:
# разобьём наблюдения на тестовую и обучающую выборки, для теста оставим 30%
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.3)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [17]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 2
нет в тестовой выборке, но есть в обучающей: 11
всего пользователей: 20465


In [18]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

C:\Users\adept\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [24]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    ".\lastfm_small\\"+test_file_name,
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    ".\lastfm_small\\"+train_file_name,
    sep="\t",
    header=False,
    index=False
)

In [25]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [34]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float64),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66799, 20466)
Ненулевых элементов в матрице: 699953
Доля ненулевых элементов: 0.0005119952695716651
Среднее значение ненулевых элементов: 216.75081898356032
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


In [39]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
sparse_info(model.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 0.5007832050323486 секунд
Размерности матрицы: (66799, 66799)
Ненулевых элементов в матрице: 1154226
Доля ненулевых элементов: 0.00025867289936011545
Среднее значение ненулевых элементов: 0.42603921157914004
Максимальное значение ненулевых элементов: 1.0000000000000049
Минимальное значение ненулевых элементов: 0.0


In [40]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(test_file_name + ".recs.tsv", "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 13.39092469215393 секунд


In [41]:
from IPython.display import Image

![title](cosine.png)

## Построим рекомендации с помощью ALS

In [38]:
model = implicit.als.AlternatingLeastSquares(factors=50)
print("строим матрицу схожести по ALS")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по ALS мере за {} секунд".format(
        time.time() - start))

строим матрицу схожести по ALS
построили матрицу схожести по ALS мере за 3.580038070678711 секунд


In [37]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(test_file_name + ".recs.tsv", "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 46.27765488624573 секунд


![title](ALS.png)